# According to wiki "sumary of lstm_speech_siamese", we can say the one shot may not be the best net structure for the problem. 

Here I try to build another net structure. 

input:
choose an anchor word, and random other words. combine all mfcc into input. the length of inputs may change. 
Output:
the one hot vector, the length will be length(input) - 1. [0, 0, 0, 1, .....]. the 1 means that words is the best match for anchor. 
the output can be fixed to 128. that means the input cannot beyond 128. in another word, the max commands is not beyond 128. 

the network

### mfcc->TimeDistributed(Dense)->gru--|
### mfcc->TimeDistributed(Dense)->gru--|
### ....................      --|-lstm(dynamic input->1024 vector)->dense(1024)->dense(256)->dense(128, sigmoid)->output
### mfcc->TimeDistributed(Dense)->gru--|
### mfcc->TimeDistributed(Dense)->gru--|

the point is how to build a dynamic lstm input. 

Keep mfcc pre-process


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import math, time, datetime
import os.path
import random
import re
import sys
import tarfile
import shutil
#print(sys.executable)
import matplotlib.pyplot as plt
import pydot
import graphviz
import numpy as np
import librosa as rosa
import librosa.display
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import numpy as np
from sklearn import preprocessing #copy from echo 1110/2018

import tensorflow as tf

from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Lambda, BatchNormalization, Activation, LSTM, GRU, concatenate, TimeDistributed
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback
from scipy.spatial import distance
from tensorflow.keras.utils import multi_gpu_model
#from tensorflow.python import debug as tf_debug
#from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
#from tensorflow.python.ops import io_ops
#from tensorflow.python.platform import gfile
#from tensorflow.python.util import compat
import local_config
default_improving_torlerence = 32 # for how many time no acc improving then terminate the training.
default_number_of_mfcc=128
default_sample_rate=16000
default_hop_length=512 
default_wav_duration=1 # 1 second
default_train_samples=10000
default_test_samples=100
default_epochs=10
default_batch_size=1024
default_sequence_size=5 # the default sequence size for how many wavs compare to anchor. 
default_train_wanted_words=["one", "two",  "backward", "bird", "cat", "dog", "five", "forward", "four", "go", "happy", "house", "learn", "left", "marvin", "nine", "no", "off", "right", "seven", "sheila", "stop", "three", "tree",  "wow", "zero","up"]
default_test_wanted_words=["bed", "eight", "visual", "follow"]



default_kernel_regularizer=regularizers.l2(0.01)
default_activity_regularizer=None #regularizers.l1(0.01)

#sess = K.get_session()
#sess = tf_debug.LocalCLIDebugWrapperSession(sess)
#K.set_session(sess)

#set backgroud noise as the base mfcc input
default_noise_background_wav=local_config.speech_data_dir+"/_background_noise_/dude_miaowing.wav"
base_loader, base_sample_rate = rosa.load(default_noise_background_wav, sr=default_sample_rate, duration=1.0)
base_mfcc = rosa.feature.mfcc(y=base_loader, sr=base_sample_rate, n_mfcc=default_number_of_mfcc)
#print(base_mfcc.shape)

### One shot keyword trigger


Here is another implementation of one-shot learning of keyword trigger with librosa mfcc. 
librosa cannot put into tensorflow graph. so mfcc computation will be done before conv network. 
that means load_wav_mfcc has to convert all wav file to mfcc vector. 
Here i have to understand
    1, what is the good mfcc vector dimension. 20, 127 may not be the right input for conv network. 
    2, even the mfcc output of librosa is not the same as tensorflow contrib.decode wav, it is enough if it has all audio feature. put librosa mfcc output as input of conv net, it will do good learning about feature abstraction. 
    3, conv net may not be that difficult. just like conv2d -> maxpooling -> conv2d->flatten->dense with softmax. 
    4, build the train network with librosa and conv net.
    5, take the dense vector output as feature extractor. 
    6, build siamese network with the feature extractor. 
    7, may add couples of dense layer to learn the feature mapping and comparation of siamese. 
    8, if that works, we get an one-shot learning for key word trigger...
    9, in reality, we still have to work out how to split the audio stream into audio clip as the input the librosa mfcc.
    

### MFCC 

extract MFCC from wav file
what is the wav parameter for MFCC output

tensorflow speech command parameter 
{'desired_samples': 16000, 'window_size_samples': 480, 'window_stride_samples': 160, 'spectrogram_length': 98, 'fingerprint_width': 40, 'fingerprint_size': 3920, 'label_count': 12, 'sample_rate': 16000, 'preprocess': 'mfcc', 'average_window_width': -1}

Mel-frequency cepstral coefficients (MFCCs)
Parameters:	
y:np.ndarray [shape=(n,)] or None
audio time series
sr:number > 0 [scalar]
sampling rate of y
S:np.ndarray [shape=(d, t)] or None
log-power Mel spectrogram
n_mfcc: int > 0 [scalar]
number of MFCCs to return
Returns:	
M:np.ndarray [shape=(n_mfcc, t)]
MFCC sequence

need more study about MFCC output


## How to calculate the lenght of mfcc vector
Short Answer

You can specify the change the length by changing the parameters used in the stft calculations. The following code will double the size of your output (20 x 113658)

data = librosa.feature.mfcc(y=y, sr=sr, n_fft=1012, hop_length=256, n_mfcc=20)
Long Answer

Librosa's librosa.feature.mfcc() function really just acts as a wrapper to librosa's librosa.feature.melspectrogram() function (which is a wrapper to librosa.core.stft and librosa.filters.mel functions).

All of the parameters pertaining to segementation of the audio signal - namely the frame and overlap values - are specified utilized in the Mel-scaled power spectrogram function (with other tune-able parameters specified for nested core functions). You specify these parameters as keyword arguments in the librosa.feature.mfcc() function.

All extra **kwargs parameters are fed to librosa.feature.melspectrogram() and subsequently to librosa.filters.mel()

By Default, the Mel-scaled power spectrogram window and hop length are the following:

n_fft=2048

hop_length=512

So assuming you used the default sample rate (sr=22050), the output of your mfcc function makes sense:

output length = (seconds) * (sample rate) / (hop_length)

(1319) * (22050) / (512) = 56804 samples


the mfcc vector size is 128 * 32   

1 * 16000/512 = 31.25 = 32 

In [ ]:

def get_default_mfcc_length(default_wav_duration=1):
    length = int(math.ceil(default_wav_duration * default_sample_rate / default_hop_length))
    return length

def load_wav_mfcc(filename):
    default_mfcc_length = get_default_mfcc_length()
    #the base_mfcc is the (128, 32) vector
    assert(base_mfcc.shape[1] == 32)
    
    out_mfcc = np.copy(base_mfcc)
    wav_loader, sample_rate = rosa.load(filename, sr=default_sample_rate)
    #print(rosa.get_duration(wav_loader, sample_rate))
    wav_mfcc = rosa.feature.mfcc(y=wav_loader, sr=default_sample_rate, n_mfcc=default_number_of_mfcc)
    out_mfcc[:, 0:wav_mfcc.shape[1]] = wav_mfcc    
    out_mfcc = np.transpose(out_mfcc)
    return out_mfcc

def mfcc_display(mfccs):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mfccs, x_axis='time')
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()
    
wav_mfcc_data = load_wav_mfcc(local_config.speech_data_dir + "/six/f6a380e7_nohash_0.wav")
print(wav_mfcc_data.shape)
mfcc_display(np.transpose(wav_mfcc_data))



# Wav MFCC loader
Wav file loader and export mfcc sequence. 

0, go throught all wav file to add background voice into command wav file
1, go through all wav file and convert to MFCC sequence
2, construct pair of MFCC sequence and a target (0 or 1, 0 for different command, 1 for the same command)
    the same word * 1000, random generate key index, the first index of wav, and the second index of wav. 
    the diff word * 1000, random generae two key index, the first index of wav, and the second index of wav. 
    the format will be [mfcc 1, mfcc 2, 0/1 for the same or different]
3, prepare pair of MFCC and targets according to batch size.  

In [ ]:
class WavMFCCLoader(object):
    def __init__(self, data_dir, wanted, validation_percentage=0, testing_percentage=0):
        self.data_dir = data_dir
        self.wanted = wanted
        self.default_mfcc_length=get_default_mfcc_length(default_wav_duration)
        self.wav_files = dict()
        self.wav_file_index()
        self.mfccs_mean = 0
        self.mfccs_std = 0
        
        
    def wav_file_index(self):
        for dirpath, dirnames, files in os.walk(self.data_dir):
            for name in files:
                if name.lower().endswith('.wav'):
                    word_name = dirpath.rsplit(local_config.default_split, 1)[1];
                    if word_name in self.wanted:
                        file_name = os.path.join(dirpath, name)
                        #print(file_name, dirpath, word_name)
    
                        if word_name in self.wav_files.keys():
                            self.wav_files[word_name].append(file_name)
                        else:
                            self.wav_files[word_name] = [file_name]
                    
        return self.wav_files
    def mfcc_preprocess(self, mfcc_data_in, preprocess="l2-normalize"):
        mfcc_data_out = np.zeros(mfcc_data_in.shape)
        if preprocess == "global":
            self.get_sample_mean_std()
            mfcc_data_out = (mfcc_data_in - self.mfccs_mean[0:mfcc_data_in.shape[0], :]) / self.mfccs_std[0:mfcc_data_in.shape[0], :]
        elif preprocess == "z-score":
            mfcc_data_out = preprocessing.scale(mfcc_data_in)
        elif preprocess == "max-min-scaler":
            mfcc_data_out = preprocessing.MinMaxScaler().fit_transform(mfcc_data_in)
        elif preprocess == "l2-normalize":
            mfcc_data_out = preprocessing.normalize(mfcc_data_in, norm='l2')
        else:
            raise ValueError("unknown proprocess")  
        return mfcc_data_out
    #sequence size here means how many wavs compare to the anchor. 
    #the output will be anchor mfcc, the sequence mfcc and one-hot vector 
    #to indicate which one in the sequence is the same word as the anchor
    def wavs_to_mfcc_sequence(self, local_preprocess="l2-normalize", sequence_size=5):
        how_many_words = len(self.wanted)
        a_wav_index = b_wav_index = -1
        #anchor mfcc
        a_index = random.randint(0, how_many_words - 1) #anchor index
        a_wav_index = random.randint(0, len(self.wav_files[self.wanted[a_index]]) - 1)
        mfcc_anchor = load_wav_mfcc(self.wav_files[self.wanted[a_index]][a_wav_index])
        mfcc_anchor = self.mfcc_preprocess(mfcc_anchor, preprocess=local_preprocess)
        #sequence of mfccs
        sequence_vec = np.zeros((sequence_size))
        mfcc_seqs = np.zeros((sequence_size, mfcc_anchor.shape[0], mfcc_anchor.shape[1]))
        #where to put the same word wav as anchor
        set_one_pos = random.randint(0, sequence_size - 1) 
        for i in range(sequence_size):
            if (i == set_one_pos):
                b_wav_index = random.randint(0, len(self.wav_files[self.wanted[a_index]]) - 1)
                mfcc_seq = load_wav_mfcc(self.wav_files[self.wanted[a_index]][b_wav_index])    
                sequence_vec[i] = 1
            else:
                b_index = random.randint(0, how_many_words - 1)
                while (a_index == b_index): # there has to be only one wav file with the same word as anchor
                    b_index = random.randint(0, how_many_words - 1)
                b_wav_index = random.randint(0, len(self.wav_files[self.wanted[b_index]]) - 1)
                mfcc_seq = load_wav_mfcc(self.wav_files[self.wanted[b_index]][b_wav_index])
                sequence_vec[i] = 0
                
            mfcc_seqs[i] = self.mfcc_preprocess(mfcc_seq, preprocess=local_preprocess)
        #mfcc_seqs = np.transpose(mfcc_seqs, (1, 2, 0))            
        #print("aaa", mfcc_1.shape, mfcc_2.shape)    
        return mfcc_anchor, mfcc_seqs, sequence_vec
        
    def get_mfcc_sequences(self, how_many, local_sequence_size = 5, preprocess='l2-normalize'):
        mfcc_anchors = np.zeros((how_many, self.default_mfcc_length, default_number_of_mfcc))
        mfcc_sequences = np.zeros((how_many, sequence_size, self.default_mfcc_length, default_number_of_mfcc))
        sequence_vecs = np.zeros((how_many, sequence_size))
        for i in range(how_many):
            
            mfcc_anchors[i], mfcc_sequences[i], sequence_vecs[i] = self.wavs_to_mfcc_sequence(sequence_size = local_sequence_size, local_preprocess = preprocess)
            
            
    
        return mfcc_anchors, mfcc_sequences, sequence_vecs
    
    def get_mfcc_for_word(self, wanted, local_preprocess='l2-normalize'):
        mfcc_data = np.zeros((self.default_mfcc_length, default_number_of_mfcc))
        if (wanted in self.wanted):
            wav_index = random.randint(0, len(self.wav_files[wanted]) - 1)
            mfcc_data_tmp = load_wav_mfcc(self.wav_files[wanted][wav_index])
            mfcc_data_tmp = self.mfcc_preprocess(mfcc_data_tmp, preprocess=local_preprocess)
            
        
            mfcc_data[0:mfcc_data_tmp.shape[0], : ] = mfcc_data_tmp
        else:
            raise ValueError("the word is not in the list")
            
        return mfcc_data, self.wav_files[wanted][wav_index]
        
    def get_sample_mean_std(self):
        count = 0
        for i in (self.wanted):
            #print(i)
            for j in range(len(self.wav_files[i])):
                #print(self.wav_files[i][j])
                count += 1
        #print("count:", count)
        mfccs = np.zeros((count, self.default_mfcc_length, default_number_of_mfcc))
        index = 0
        for i in (self.wanted):
            for j in range(len(self.wav_files[i])):       
                mfcc_ = load_wav_mfcc(self.wav_files[i][j])
                mfccs[index, 0:mfcc_.shape[0], : ] = mfcc_
                index += 1
        self.mfccs_mean = mfccs.mean(0)
        self.mfccs_std = mfccs.std(0)
        

#loader = WavMFCCLoader(local_config.speech_data_dir, wanted=["one", "two", "bed", "backward", "bird", "cat", "dog", "eight", "five", "follow", "forward", "four", "go", "happy", "house", "learn", "left", "marvin", "nine", "no", "off", "right", "seven", "sheila", "stop", "three", "tree", "visual", "wow", "zero"])
loader = WavMFCCLoader(local_config.speech_data_dir, wanted=["one", "two", "bed", "backward"])
sequence_size = 5
#wav_list = loader.wav_file_index()
mfcc_anchors, mfcc_sequences, sequence_vecs = loader.get_mfcc_sequences(3, local_sequence_size=sequence_size, preprocess='l2-normalize')
for i in range(3):
    mfcc_display(np.transpose(mfcc_anchors[i]))
    print("the " + str(i) + " seq vec")
    for j in range(sequence_size):
        mfcc_display(np.transpose(mfcc_sequences[i][j]))
        print(sequence_vecs[i][j])
        

## Data loader for training and testing 

In [ ]:

def read_data_file(file_name):
    f = open(file_name, "rb")
    t = np.load(f)
    #print("shape:", t.shape)
    f.close()
    return t

def write_data_file(file_name, data):
    f = open(file_name, "wb")
    np.save(f, data)
    f.close()

def save_model_file(model):
    json_model = model.to_json()    
    with open(local_config.default_processed_data_path + local_config.default_split + "model_description.txt", "w") as out_file:
        out_file.write(json_model)
        out_file.close()
    plot_model(model, to_file=local_config.default_processed_data_path + local_config.default_split + 'model_description.png')
    

def store_processed_data(local_train_samples=default_train_samples, sequence_size = default_sequence_size,local_test_samples=default_test_samples, local_process_data_path=local_config.default_processed_data_path, local_preprocess='l2-normalize', local_train_wanted=default_train_wanted_words, local_test_wanted=default_test_wanted_words):
    train_loader = WavMFCCLoader(local_config.speech_data_dir, wanted=local_train_wanted)
    mfcc_train_anchors, mfcc_train_seqs, train_seq_vecs = train_loader.get_mfcc_sequences(local_train_samples, local_sequence_size = sequence_size, preprocess=local_preprocess)
    mfcc_eval_anchors, mfcc_eval_seqs, eval_seq_vecs = train_loader.get_mfcc_sequences(local_test_samples, local_sequence_size = sequence_size, preprocess=local_preprocess)

    test_loader  = WavMFCCLoader(local_config.speech_data_dir, wanted=local_test_wanted)
    mfcc_test_anchors, mfcc_test_seqs, test_seq_vecs = test_loader.get_mfcc_sequences(local_test_samples, local_sequence_size = sequence_size, preprocess=local_preprocess)

    #store the training mfcc anchor to np file
    filename_train_mfcc_anchors = local_process_data_path + local_config.default_split + "mfcc_train_anchors_" + str(sequence_size) + '_' + str(local_train_samples) + "_" + local_preprocess + ".npy"
    write_data_file(filename_train_mfcc_anchors, mfcc_train_anchors)
    
    #store the training mfcc sequences to np file
    filename_train_mfcc_sequences = local_process_data_path + local_config.default_split + "mfcc_train_sequences_" + str(sequence_size) + '_'  + str(local_train_samples) + "_" + local_preprocess + ".npy"
    write_data_file(filename_train_mfcc_sequences, mfcc_train_seqs)
    
    #store the training sequence vector data to np file
    filename_train_seq_vecs = local_process_data_path + local_config.default_split + "seq_vecs_train_" + str(sequence_size) + '_'  + str(local_train_samples) + "_" + local_preprocess + ".npy"
    write_data_file(filename_train_seq_vecs, train_seq_vecs)
    
    #store the eval mfcc anchor to np file
    filename_test_mfcc_anchors = local_process_data_path + local_config.default_split + "mfcc_test_anchors_" + str(sequence_size) + '_' + str(local_test_samples) + "_" + local_preprocess + ".npy"
    write_data_file(filename_test_mfcc_anchors, mfcc_test_anchors)
    
    #store the eval mfcc sequences to np file
    filename_test_mfcc_sequences = local_process_data_path + local_config.default_split + "mfcc_test_sequences_" + str(sequence_size) + '_'  + str(local_test_samples) + "_" + local_preprocess + ".npy"
    write_data_file(filename_test_mfcc_sequences, mfcc_test_seqs)
    
    #store the eval sequence vector data to np file
    filename_test_seq_vecs = local_process_data_path + local_config.default_split + "seq_vecs_test_" + str(sequence_size) + '_'  + str(local_test_samples) + "_" + local_preprocess + ".npy"
    write_data_file(filename_test_seq_vecs, test_seq_vecs)
    
    #store the eval mfcc anchor to np file
    filename_eval_mfcc_anchors = local_process_data_path + local_config.default_split + "mfcc_eval_anchors_" + str(sequence_size) + '_' + str(local_test_samples) + "_" + local_preprocess + ".npy"
    write_data_file(filename_eval_mfcc_anchors, mfcc_eval_anchors)
    
    #store the eval mfcc sequences to np file
    filename_eval_mfcc_sequences = local_process_data_path + local_config.default_split + "mfcc_eval_sequences_" + str(sequence_size) + '_'  + str(local_test_samples) + "_" + local_preprocess + ".npy"
    write_data_file(filename_eval_mfcc_sequences, mfcc_eval_seqs)
    
    #store the eval sequence vector data to np file
    filename_eval_seq_vecs = local_process_data_path + local_config.default_split + "seq_vecs_eval_" + str(sequence_size) + '_'  + str(local_test_samples) + "_" + local_preprocess + ".npy"
    write_data_file(filename_eval_seq_vecs, eval_seq_vecs)
    
    
def load_processed_data(local_train_samples=default_train_samples, local_test_samples=default_test_samples, local_process_data_path=local_config.default_processed_data_path, sequence_size = default_sequence_size, local_preprocess='l2-normalize'):
    
    filename_train_mfcc_anchor = local_process_data_path + local_config.default_split + "mfcc_train_anchors_" + str(sequence_size) + '_'  + str(local_train_samples) + "_" + local_preprocess + ".npy"    
    
    if not (os.path.isfile(filename_train_mfcc_anchor)):
        store_processed_data(local_train_samples, sequence_size, local_test_samples, local_process_data_path, local_preprocess)
    
    filename_train_mfcc_anchors = local_process_data_path + local_config.default_split + "mfcc_train_anchors_" + str(sequence_size) + '_' + str(local_train_samples) + "_" + local_preprocess + ".npy"
    mfcc_train_anchors = read_data_file(filename_train_mfcc_anchors)
    filename_train_mfcc_sequences = local_process_data_path + local_config.default_split + "mfcc_train_sequences_" + str(sequence_size) + '_' + str(local_train_samples) + "_" + local_preprocess + ".npy"
    mfcc_train_sequences = read_data_file(filename_train_mfcc_sequences)
    filename_train_seq_vecs = local_process_data_path + local_config.default_split + "seq_vecs_train_" + str(sequence_size) + '_' + str(local_train_samples) + "_" + local_preprocess + ".npy"
    train_seq_vecs = read_data_file(filename_train_seq_vecs)
    filename_eval_mfcc_anchors = local_process_data_path + local_config.default_split + "mfcc_eval_anchors_" + str(sequence_size) + '_' + str(local_test_samples) + "_" + local_preprocess + ".npy"
    mfcc_eval_anchors = read_data_file(filename_eval_mfcc_anchors)
    filename_eval_mfcc_sequences = local_process_data_path + local_config.default_split + "mfcc_eval_sequences_" + str(sequence_size) + '_' + str(local_test_samples) + "_" + local_preprocess + ".npy"
    mfcc_eval_sequences = read_data_file(filename_eval_mfcc_sequences)
    filename_eval_seq_vecs = local_process_data_path + local_config.default_split + "seq_vecs_eval_" + str(sequence_size) + '_' + str(local_test_samples) + "_" + local_preprocess + ".npy"
    eval_seq_vecs = read_data_file(filename_eval_seq_vecs)
    filename_test_mfcc_anchors = local_process_data_path + local_config.default_split + "mfcc_test_anchors_" + str(sequence_size) + '_' + str(local_test_samples) + "_" + local_preprocess + ".npy"
    mfcc_test_anchors = read_data_file(filename_test_mfcc_anchors)
    filename_test_mfcc_sequences = local_process_data_path + local_config.default_split + "mfcc_test_sequences_" + str(sequence_size) + '_' + str(local_test_samples) + "_" + local_preprocess + ".npy"
    mfcc_test_sequences = read_data_file(filename_test_mfcc_sequences)
    filename_test_seq_vecs = local_process_data_path + local_config.default_split + "seq_vecs_test_" + str(sequence_size) + '_' + str(local_test_samples) + "_" + local_preprocess + ".npy"
    test_seq_vecs = read_data_file(filename_test_seq_vecs)    
    
    return mfcc_train_anchors, mfcc_train_sequences, train_seq_vecs, mfcc_eval_anchors, mfcc_eval_sequences, eval_seq_vecs, mfcc_test_anchors, mfcc_test_sequences, test_seq_vecs

#load_processed_data(local_train_samples=10000, local_test_samples=100, local_preprocess='z-score')
_=load_processed_data(local_train_samples=10000, sequence_size = 5, local_test_samples=100, local_preprocess='l2-normalize')
#load_processed_data(local_train_samples=10000, local_test_samples=100, local_preprocess='global')
#load_processed_data(local_train_samples=10000, local_test_samples=100, local_preprocess='max-min-scaler')

# LSTM Network

create a keras lstm network, take mfcc vector as input.

the speech command mfcc input shape is (?, mfcc_number, hop_number, 1)




In [ ]:
def create_lstm_model(local_input_shape, is_training=True):
    model = Sequential()
    model.add(TimeDistributed(Dense(128, kernel_regularizer=default_kernel_regularizer, activity_regularizer=default_activity_regularizer, name='time_dense'), input_shape=local_input_shape))
    model.add(GRU(128, return_sequences=False, stateful=False))
    #model.add(GRU(256, return_sequences=True, stateful=False))
    #model.add(GRU(256, stateful=False))

    model.add(Dense(128, kernel_regularizer=default_kernel_regularizer, activity_regularizer=default_activity_regularizer, name='after_gru_dense'))
    model.add(BatchNormalization())
    model.add(Activation("relu")) 
    #if (is_training):
    #    model.add(Dropout(0.5))
    #model.add(Dense(labels_count, activation="softmax"))
    
    return model


def euclidean_distance(x):     
    ret = K.mean( K.square(x[0]-x[1]),axis=-1,keepdims=True)
    #print(ret.shape)
    return ret #K.mean( K.square(x1-x2),axis=-1,keepdims=True)
    #return distance.euclidean(x[0], x[1])

def cosine_distance(x):        
    x1 = K.l2_normalize(x[0], axis=-1)
    x2 = K.l2_normalize(x[1], axis=-1)
    return -K.mean(x1 * x2, axis=-1, keepdims=True)

def create_siamese_model(input_shape, sequence_size = default_sequence_size, siamese_mode = 'abs'):
    
    anchor_input = Input(input_shape, name='anchor_input')
    keras_model = create_lstm_model(input_shape)
    anchor_encoder = keras_model(anchor_input)
    seq_inputs = [anchor_input]
    seq_inputs_encoder = []
    distance_vector = []
    for i in range(sequence_size):
        seq_input = Input(input_shape, name='seq'+str(i)+'_input')
        seq_encoder = keras_model(seq_input)
        if (siamese_mode == 'dot'):
            calculate_layer = Lambda(lambda x: x[0]*x[1], name='lambda_dot'+str(i))([anchor_encoder, seq_encoder])
        elif (siamese_mode == 'abs'):
            calculate_layer = Lambda(lambda x: tf.abs(x[0]-x[1], name='lambda_abs'+str(i)))([anchor_encoder, seq_encoder])
        elif (siamese_mode == 'euclidean'):
            calculate_layer = Lambda(euclidean_distance, name='lambda_euclidean'+str(i))([anchor_encoder, seq_encoder])
        elif (siamese_mode == 'cosine'):
            calculate_layer = Lambda(cosine_distance, name='lambda_cosine'+str(i))([anchor_encoder, seq_encoder])
        else:
            raise ValueError("unknown siamese_mode")   
        seq_inputs.append(seq_input)
        seq_inputs_encoder.append(calculate_layer)
        
   
    concatenated_layer = concatenate(inputs=seq_inputs_encoder)
    
        
    if (siamese_mode == 'dot') or (siamese_mode == 'abs'):     
        last_hiden_layer = Dense(512, activation='relu', kernel_regularizer=default_kernel_regularizer, activity_regularizer=default_activity_regularizer, name='dense_512')(concatenated_layer)
    else:
        last_hiden_layer = concatenated_layer
    output_layer = Dense(sequence_size, activation='softmax', kernel_regularizer=default_kernel_regularizer, activity_regularizer=default_activity_regularizer, name='dense_output')(last_hiden_layer)
    
    siamese_model = Model(seq_inputs, output_layer)
    return siamese_model
    


## Training and Testing

In [ ]:
class EarlyStopOnBaseline(Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, monitor='val_acc', baseline=0.9, patience=0):
        super(EarlyStopOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline
        self.patience = patience
        self.count_for_patience = 0
        self.count_for_best = 0
        self.best_acc = 0
        self.torlerence = default_improving_torlerence

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        #print('Epoch %d: EarlyStopOnBaseline call back'%epoch)
        if acc is not None:
            if acc >= self.baseline:
                self.count_for_patience = self.count_for_patience + 1
                if (self.count_for_patience >= self.patience):
                    print('Epoch %d: Reached baseline, terminating training' % (epoch))
                    self.model.stop_training = True
            elif acc > self.best_acc:
                self.best_acc = acc
                self.count_for_best = 0 
                #print(self.best_acc)
            elif acc <= self.best_acc:
                self.count_for_best = self.count_for_best + 1
                #print(self.count_for_best)
                if self.count_for_best >= self.torlerence:
                    print('Epoch %d: No improving for long time, terminating training' % (epoch))
                    self.model.stop_training = True

def siamese_train(mfcc_train_anchors, mfcc_train_sequences, train_seq_vecs, mfcc_eval_anchors, mfcc_eval_sequences, eval_seq_vecs, local_sequence_size=default_sequence_size,local_siamese_mode='abs', local_batch_size=default_batch_size, local_epochs= default_epochs, local_optimizer='adam'):
    default_mfcc_length = get_default_mfcc_length(default_wav_duration)
    siamese_model = create_siamese_model((default_mfcc_length, default_number_of_mfcc), sequence_size=local_sequence_size, siamese_mode=local_siamese_mode)
    #save_model_file(siamese_model)
    #siamese_model = multi_gpu_model(siamese_model, gpus=2)
    #siamese_model.summary()
    siamese_model.compile(loss='categorical_crossentropy', optimizer=local_optimizer, metrics=['accuracy'])
    x_train_inputs=[mfcc_train_anchors]
    mfcc_train_sequences = np.transpose(mfcc_train_sequences, (1, 0, 2, 3))
    for i in range(sequence_size): 
        
        #print(mfcc_train_sequences.shape)
        x_train_inputs.append(mfcc_train_sequences[i])
    y_train = train_seq_vecs

    #callbacks , , histogram_freq=32
    tensorboard = keras.callbacks.TensorBoard(log_dir=local_config.default_processed_data_path+'/logs', write_graph=True, write_grads=True, write_images=True)
    earlystop = EarlyStopOnBaseline(monitor='val_acc',baseline=0.8)
    #modelchecker = keras.callbacks.ModelCheckpoint(local_config.default_model_path, monitor='val_loss', verbose=0, save_best_only=True)
    #print("x_train_inputs " + str(x_train_inputs[0].shape) + " y_train " + str(y_train))
    siamese_model.fit(x_train_inputs, y_train, epochs=local_epochs, batch_size=local_batch_size, callbacks=[tensorboard, earlystop], validation_split=0.2) #, earlystop
    
    
    x_eval_inputs=[mfcc_eval_anchors]
    mfcc_eval_sequences = np.transpose(mfcc_eval_sequences, (1, 0, 2, 3))
    for i in range(sequence_size):        
        x_eval_inputs.append(mfcc_eval_sequences[i])
    y_eval = eval_seq_vecs

    
    loss, accuracy = siamese_model.evaluate(x_eval_inputs, y_eval)    
    print(str(loss)+'  ' +str(accuracy))
    siamese_model.save(local_config.default_model_path+"/speech_siamese.h5")

    
    return accuracy

def siamese_test(mfcc_test_anchors, mfcc_test_sequences, test_seq_vecs):
    default_mfcc_length = get_default_mfcc_length(default_wav_duration)    
    siamese_model = keras.models.load_model(local_config.default_model_path+"/speech_siamese.h5")    

    x_test_inputs=[mfcc_test_anchors]
    mfcc_test_sequences = np.transpose(mfcc_test_sequences, (1, 0, 2, 3))
    for i in range(sequence_size):
        
        x_test_inputs.append(mfcc_test_sequences[i])
    y_test = test_seq_vecs
    
    loss, accuracy = siamese_model.test_on_batch(x=x_test_inputs, y=y_test)
    print(loss)
    
    #keep the accurcy beyond .75 to a separated file
    if accuracy > .75:
        good_model_filename = local_config.default_model_path+ "/speech_siamese_"+str(int(accuracy * 100))+".h5"
        shutil.copyfile(local_config.default_model_path+"/speech_siamese.h5", good_model_filename)
        
    return accuracy
    

## main 

In [ ]:
#wav_mfcc = load_wav_mfcc("/Users/hermitwang/Downloads/speech_dataset/backward/0a2b400e_nohash_0.wav")
#print(wav_mfcc.shape) 
train_samples = default_train_samples #* 5
test_samples = default_test_samples
batch_size_numbers=[128, 256, 512, 1024, 2048]
optimizers = ['adam', 'nadam', 'adamax']
siamese_modes = ['abs', 'dot' ]
train_epochs = 500
print("Start at " + str(datetime.datetime.now()))
mfcc_train_anchors, mfcc_train_sequences, train_seq_vecs, mfcc_eval_anchors, mfcc_eval_sequences, eval_seq_vecs, mfcc_test_anchors, mfcc_test_sequences, test_seq_vecs = load_processed_data(local_train_samples=train_samples, sequence_size = 5, local_test_samples=test_samples, local_preprocess='l2-normalize')

for i in range(10):
    j = 3
    for m in range(len(optimizers)):
        for n in range(len(siamese_modes)):
            print("Train batch size " + str(batch_size_numbers[j]) + " of " + str(i+1) + " run with optimizer " + optimizers[m] + " and siamese mode "+siamese_modes[n]+" start at " + str(datetime.datetime.now()))
            score=siamese_train(mfcc_train_anchors, mfcc_train_sequences, train_seq_vecs, mfcc_eval_anchors, mfcc_eval_sequences, eval_seq_vecs, local_sequence_size=default_sequence_size, local_siamese_mode=siamese_modes[n], local_batch_size=batch_size_numbers[j], local_epochs=train_epochs, local_optimizer=optimizers[m])
            print(score)
            score=siamese_test(mfcc_test_anchors, mfcc_test_sequences, test_seq_vecs)
    
            print(score)

# Test on group data 

the real case to use the model is compare the input to all prebuilt command. so if everytime the matched command get the highest possibility, the model is feasible enough.

the follow code is trying to evaluate the model. 

# siamese mode abs and dot

siamese mode: abs

Start at 2018-12-18 19:59:19.594174
Train batch size 1024 of 1 run start at 2018-12-18 19:59:20.364893:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
anchor_input (InputLayer)       (None, 32, 128)      0                                            
__________________________________________________________________________________________________
seq0_input (InputLayer)         (None, 32, 128)      0                                            
__________________________________________________________________________________________________
seq1_input (InputLayer)         (None, 32, 128)      0                                            
__________________________________________________________________________________________________
seq2_input (InputLayer)         (None, 32, 128)      0                                            
__________________________________________________________________________________________________
seq3_input (InputLayer)         (None, 32, 128)      0                                            
__________________________________________________________________________________________________
seq4_input (InputLayer)         (None, 32, 128)      0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 128)          132224      anchor_input[0][0]               
                                                                 seq0_input[0][0]                 
                                                                 seq1_input[0][0]                 
                                                                 seq2_input[0][0]                 
                                                                 seq3_input[0][0]                 
                                                                 seq4_input[0][0]                 
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 128)          0           sequential[1][0]                 
                                                                 sequential[2][0]                 
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128)          0           sequential[1][0]                 
                                                                 sequential[3][0]                 
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 128)          0           sequential[1][0]                 
                                                                 sequential[4][0]                 
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 128)          0           sequential[1][0]                 
                                                                 sequential[5][0]                 
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 128)          0           sequential[1][0]                 
                                                                 sequential[6][0]                 
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 640)          0           lambda[0][0]                     
                                                                 lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
                                                                 lambda_3[0][0]                   
                                                                 lambda_4[0][0]                   
__________________________________________________________________________________________________
dense_512 (Dense)               (None, 512)          328192      concatenate[0][0]                
__________________________________________________________________________________________________
dense_output (Dense)            (None, 5)            2565        dense_512[0][0]                  
==================================================================================================
Total params: 462,981
Trainable params: 462,725
Non-trainable params: 256
__________________________________________________________________________________________________
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 8s 1ms/step - loss: 8.7885 - acc: 0.2041 - val_loss: 7.4398 - val_acc: 0.2015
Epoch 2/100
8000/8000 [==============================] - 4s 470us/step - loss: 6.7969 - acc: 0.2260 - val_loss: 6.0289 - val_acc: 0.2055
Epoch 3/100
8000/8000 [==============================] - 4s 455us/step - loss: 5.5353 - acc: 0.2469 - val_loss: 4.9495 - val_acc: 0.2095
Epoch 4/100
8000/8000 [==============================] - 4s 454us/step - loss: 4.5581 - acc: 0.2451 - val_loss: 4.1019 - val_acc: 0.1985
Epoch 5/100
8000/8000 [==============================] - 4s 463us/step - loss: 3.7969 - acc: 0.2361 - val_loss: 3.4441 - val_acc: 0.2070
Epoch 6/100
8000/8000 [==============================] - 4s 462us/step - loss: 3.2052 - acc: 0.2465 - val_loss: 2.9408 - val_acc: 0.2050
Epoch 7/100
8000/8000 [==============================] - 4s 469us/step - loss: 2.7643 - acc: 0.2420 - val_loss: 2.5683 - val_acc: 0.1955
Epoch 8/100
8000/8000 [==============================] - 4s 460us/step - loss: 2.4387 - acc: 0.2399 - val_loss: 2.2987 - val_acc: 0.2020
Epoch 9/100
8000/8000 [==============================] - 4s 464us/step - loss: 2.2053 - acc: 0.2338 - val_loss: 2.1059 - val_acc: 0.2095
Epoch 10/100
8000/8000 [==============================] - 4s 464us/step - loss: 2.0406 - acc: 0.2156 - val_loss: 1.9684 - val_acc: 0.2140
Epoch 11/100
8000/8000 [==============================] - 4s 466us/step - loss: 1.9215 - acc: 0.2288 - val_loss: 1.8707 - val_acc: 0.2110
Epoch 12/100
8000/8000 [==============================] - 4s 460us/step - loss: 1.8334 - acc: 0.2349 - val_loss: 1.7993 - val_acc: 0.2405
Epoch 13/100
8000/8000 [==============================] - 4s 466us/step - loss: 1.7704 - acc: 0.2535 - val_loss: 1.7409 - val_acc: 0.2585
Epoch 14/100
8000/8000 [==============================] - 4s 462us/step - loss: 1.6972 - acc: 0.2840 - val_loss: 1.6573 - val_acc: 0.3070
Epoch 15/100
8000/8000 [==============================] - 4s 444us/step - loss: 1.6398 - acc: 0.3040 - val_loss: 1.6329 - val_acc: 0.3000
Epoch 16/100
8000/8000 [==============================] - 4s 461us/step - loss: 1.6009 - acc: 0.3179 - val_loss: 1.6026 - val_acc: 0.3235
Epoch 17/100
8000/8000 [==============================] - 4s 448us/step - loss: 1.5853 - acc: 0.3275 - val_loss: 1.6148 - val_acc: 0.3245
Epoch 18/100
8000/8000 [==============================] - 3s 437us/step - loss: 1.5493 - acc: 0.3419 - val_loss: 1.6030 - val_acc: 0.3110
Epoch 19/100
8000/8000 [==============================] - 4s 458us/step - loss: 1.5386 - acc: 0.3440 - val_loss: 1.5772 - val_acc: 0.3190
Epoch 20/100
8000/8000 [==============================] - 4s 460us/step - loss: 1.5350 - acc: 0.3455 - val_loss: 1.6079 - val_acc: 0.3090
Epoch 21/100
8000/8000 [==============================] - 4s 460us/step - loss: 1.5255 - acc: 0.3491 - val_loss: 1.5841 - val_acc: 0.3355
Epoch 22/100
8000/8000 [==============================] - 4s 465us/step - loss: 1.5207 - acc: 0.3515 - val_loss: 1.5829 - val_acc: 0.3260
Epoch 23/100
8000/8000 [==============================] - 4s 463us/step - loss: 1.5109 - acc: 0.3534 - val_loss: 1.5835 - val_acc: 0.3240
Epoch 24/100
8000/8000 [==============================] - 4s 462us/step - loss: 1.5146 - acc: 0.3575 - val_loss: 1.5696 - val_acc: 0.3160
Epoch 25/100
8000/8000 [==============================] - 4s 467us/step - loss: 1.5089 - acc: 0.3515 - val_loss: 1.5706 - val_acc: 0.3250
Epoch 26/100
8000/8000 [==============================] - 4s 454us/step - loss: 1.5003 - acc: 0.3700 - val_loss: 1.5799 - val_acc: 0.3365
Epoch 27/100
8000/8000 [==============================] - 4s 462us/step - loss: 1.5000 - acc: 0.3602 - val_loss: 1.5167 - val_acc: 0.3520
Epoch 28/100
8000/8000 [==============================] - 4s 459us/step - loss: 1.4883 - acc: 0.3678 - val_loss: 1.6139 - val_acc: 0.3020
Epoch 29/100
8000/8000 [==============================] - 4s 469us/step - loss: 1.4940 - acc: 0.3610 - val_loss: 1.5333 - val_acc: 0.3310
Epoch 30/100
8000/8000 [==============================] - 4s 468us/step - loss: 1.4799 - acc: 0.3737 - val_loss: 1.5885 - val_acc: 0.3240
Epoch 31/100
8000/8000 [==============================] - 4s 453us/step - loss: 1.4781 - acc: 0.3670 - val_loss: 1.5658 - val_acc: 0.3245
Epoch 32/100
8000/8000 [==============================] - 4s 461us/step - loss: 1.4900 - acc: 0.3634 - val_loss: 1.5249 - val_acc: 0.3565
Epoch 33/100
8000/8000 [==============================] - 4s 459us/step - loss: 1.4744 - acc: 0.3811 - val_loss: 1.5313 - val_acc: 0.3485
Epoch 34/100
8000/8000 [==============================] - 4s 464us/step - loss: 1.4754 - acc: 0.3811 - val_loss: 1.5567 - val_acc: 0.3340
Epoch 35/100
8000/8000 [==============================] - 4s 469us/step - loss: 1.4680 - acc: 0.3806 - val_loss: 1.5900 - val_acc: 0.3055
Epoch 36/100
8000/8000 [==============================] - 4s 456us/step - loss: 1.4695 - acc: 0.3810 - val_loss: 1.5284 - val_acc: 0.3715
Epoch 37/100
8000/8000 [==============================] - 4s 458us/step - loss: 1.4510 - acc: 0.3965 - val_loss: 1.4816 - val_acc: 0.3735
Epoch 38/100
8000/8000 [==============================] - 4s 456us/step - loss: 1.4555 - acc: 0.3943 - val_loss: 1.4506 - val_acc: 0.3920
Epoch 39/100
8000/8000 [==============================] - 4s 459us/step - loss: 1.4444 - acc: 0.4025 - val_loss: 1.4953 - val_acc: 0.3675
Epoch 40/100
8000/8000 [==============================] - 4s 458us/step - loss: 1.4472 - acc: 0.3997 - val_loss: 1.4724 - val_acc: 0.3795
Epoch 41/100
8000/8000 [==============================] - 4s 456us/step - loss: 1.4094 - acc: 0.4235 - val_loss: 1.4512 - val_acc: 0.3990
Epoch 42/100
8000/8000 [==============================] - 4s 457us/step - loss: 1.4032 - acc: 0.4280 - val_loss: 1.4430 - val_acc: 0.3995
Epoch 43/100
8000/8000 [==============================] - 4s 459us/step - loss: 1.3861 - acc: 0.4371 - val_loss: 1.3884 - val_acc: 0.4355
Epoch 44/100
8000/8000 [==============================] - 4s 457us/step - loss: 1.3681 - acc: 0.4494 - val_loss: 1.3686 - val_acc: 0.4495
Epoch 45/100
8000/8000 [==============================] - 4s 455us/step - loss: 1.3459 - acc: 0.4704 - val_loss: 1.3893 - val_acc: 0.4380
Epoch 46/100
8000/8000 [==============================] - 4s 466us/step - loss: 1.3286 - acc: 0.4711 - val_loss: 1.3520 - val_acc: 0.4565
Epoch 47/100
8000/8000 [==============================] - 4s 466us/step - loss: 1.3226 - acc: 0.4759 - val_loss: 1.3741 - val_acc: 0.4560
Epoch 48/100
8000/8000 [==============================] - 4s 468us/step - loss: 1.3032 - acc: 0.4886 - val_loss: 1.3817 - val_acc: 0.4520
Epoch 49/100
8000/8000 [==============================] - 4s 468us/step - loss: 1.2955 - acc: 0.4926 - val_loss: 1.3178 - val_acc: 0.4825
Epoch 50/100
8000/8000 [==============================] - 4s 460us/step - loss: 1.2784 - acc: 0.5064 - val_loss: 1.3889 - val_acc: 0.4460
Epoch 51/100
8000/8000 [==============================] - 4s 466us/step - loss: 1.2688 - acc: 0.5110 - val_loss: 1.2953 - val_acc: 0.4890
Epoch 52/100
8000/8000 [==============================] - 4s 459us/step - loss: 1.2570 - acc: 0.5164 - val_loss: 1.3195 - val_acc: 0.4700
Epoch 53/100
8000/8000 [==============================] - 4s 455us/step - loss: 1.2416 - acc: 0.5271 - val_loss: 1.2476 - val_acc: 0.5220
Epoch 54/100
8000/8000 [==============================] - 4s 461us/step - loss: 1.2286 - acc: 0.5373 - val_loss: 1.2951 - val_acc: 0.4900
Epoch 55/100
8000/8000 [==============================] - 4s 458us/step - loss: 1.1924 - acc: 0.5570 - val_loss: 1.2766 - val_acc: 0.5015
Epoch 56/100
8000/8000 [==============================] - 4s 462us/step - loss: 1.2035 - acc: 0.5545 - val_loss: 1.2285 - val_acc: 0.5340
Epoch 57/100
8000/8000 [==============================] - 4s 455us/step - loss: 1.1657 - acc: 0.5739 - val_loss: 1.3054 - val_acc: 0.4990
Epoch 58/100
8000/8000 [==============================] - 4s 459us/step - loss: 1.1601 - acc: 0.5776 - val_loss: 1.2178 - val_acc: 0.5375
Epoch 59/100
8000/8000 [==============================] - 4s 466us/step - loss: 1.1500 - acc: 0.5762 - val_loss: 1.2132 - val_acc: 0.5450
Epoch 60/100
8000/8000 [==============================] - 4s 463us/step - loss: 1.1244 - acc: 0.5950 - val_loss: 1.1985 - val_acc: 0.5505
Epoch 61/100
8000/8000 [==============================] - 4s 466us/step - loss: 1.1214 - acc: 0.5920 - val_loss: 1.1786 - val_acc: 0.5720
Epoch 62/100
8000/8000 [==============================] - 4s 469us/step - loss: 1.1056 - acc: 0.6100 - val_loss: 1.1585 - val_acc: 0.5725
Epoch 63/100
8000/8000 [==============================] - 4s 475us/step - loss: 1.0828 - acc: 0.6126 - val_loss: 1.2358 - val_acc: 0.5585
Epoch 64/100
8000/8000 [==============================] - 4s 474us/step - loss: 1.0678 - acc: 0.6226 - val_loss: 1.2021 - val_acc: 0.5700
Epoch 65/100
8000/8000 [==============================] - 4s 468us/step - loss: 1.0711 - acc: 0.6216 - val_loss: 1.1817 - val_acc: 0.5845
Epoch 66/100
8000/8000 [==============================] - 4s 465us/step - loss: 1.0703 - acc: 0.6199 - val_loss: 1.2144 - val_acc: 0.5730
Epoch 67/100
8000/8000 [==============================] - 4s 464us/step - loss: 1.0316 - acc: 0.6500 - val_loss: 1.1518 - val_acc: 0.5825
Epoch 68/100
8000/8000 [==============================] - 4s 468us/step - loss: 1.0159 - acc: 0.6489 - val_loss: 1.1887 - val_acc: 0.5765
Epoch 69/100
8000/8000 [==============================] - 4s 468us/step - loss: 1.0365 - acc: 0.6482 - val_loss: 1.1541 - val_acc: 0.5900
Epoch 70/100
8000/8000 [==============================] - 4s 467us/step - loss: 1.0152 - acc: 0.6589 - val_loss: 1.1209 - val_acc: 0.6095
Epoch 71/100
8000/8000 [==============================] - 4s 472us/step - loss: 0.9859 - acc: 0.6684 - val_loss: 1.1416 - val_acc: 0.5995
Epoch 72/100
8000/8000 [==============================] - 4s 467us/step - loss: 0.9886 - acc: 0.6635 - val_loss: 1.0997 - val_acc: 0.6055
Epoch 73/100
8000/8000 [==============================] - 4s 465us/step - loss: 0.9779 - acc: 0.6735 - val_loss: 1.1311 - val_acc: 0.6040
Epoch 74/100
8000/8000 [==============================] - 4s 458us/step - loss: 0.9715 - acc: 0.6762 - val_loss: 1.1223 - val_acc: 0.6120
Epoch 75/100
8000/8000 [==============================] - 4s 458us/step - loss: 0.9513 - acc: 0.6858 - val_loss: 1.1395 - val_acc: 0.5975
Epoch 76/100
8000/8000 [==============================] - 4s 464us/step - loss: 0.9561 - acc: 0.6819 - val_loss: 1.0968 - val_acc: 0.6280
Epoch 77/100
8000/8000 [==============================] - 4s 465us/step - loss: 0.9386 - acc: 0.6955 - val_loss: 1.0971 - val_acc: 0.6120
Epoch 78/100
8000/8000 [==============================] - 4s 466us/step - loss: 0.9248 - acc: 0.7054 - val_loss: 1.0546 - val_acc: 0.6500
Epoch 79/100
8000/8000 [==============================] - 4s 462us/step - loss: 0.9217 - acc: 0.7030 - val_loss: 1.0317 - val_acc: 0.6565
Epoch 80/100
8000/8000 [==============================] - 4s 467us/step - loss: 0.9028 - acc: 0.7112 - val_loss: 1.0539 - val_acc: 0.6435
Epoch 81/100
8000/8000 [==============================] - 4s 463us/step - loss: 0.8822 - acc: 0.7229 - val_loss: 1.1169 - val_acc: 0.6255
Epoch 82/100
8000/8000 [==============================] - 4s 464us/step - loss: 0.8788 - acc: 0.7204 - val_loss: 1.0404 - val_acc: 0.6420
Epoch 83/100
8000/8000 [==============================] - 4s 462us/step - loss: 0.8876 - acc: 0.7182 - val_loss: 1.0103 - val_acc: 0.6550
Epoch 84/100
8000/8000 [==============================] - 4s 463us/step - loss: 0.8603 - acc: 0.7297 - val_loss: 0.9879 - val_acc: 0.6760
Epoch 85/100
8000/8000 [==============================] - 4s 460us/step - loss: 0.8375 - acc: 0.7446 - val_loss: 1.1248 - val_acc: 0.6330
Epoch 86/100
8000/8000 [==============================] - 4s 463us/step - loss: 0.8500 - acc: 0.7381 - val_loss: 0.9894 - val_acc: 0.6865
Epoch 87/100
8000/8000 [==============================] - 4s 461us/step - loss: 0.8466 - acc: 0.7403 - val_loss: 1.0206 - val_acc: 0.6585
Epoch 88/100
8000/8000 [==============================] - 4s 461us/step - loss: 0.8149 - acc: 0.7580 - val_loss: 0.9566 - val_acc: 0.6900
Epoch 89/100
8000/8000 [==============================] - 4s 458us/step - loss: 0.7948 - acc: 0.7650 - val_loss: 0.9831 - val_acc: 0.6780
Epoch 90/100
8000/8000 [==============================] - 4s 464us/step - loss: 0.8038 - acc: 0.7625 - val_loss: 0.9769 - val_acc: 0.6860
Epoch 91/100
8000/8000 [==============================] - 4s 465us/step - loss: 0.8032 - acc: 0.7611 - val_loss: 0.9174 - val_acc: 0.7120
Epoch 92/100
8000/8000 [==============================] - 4s 466us/step - loss: 0.7685 - acc: 0.7810 - val_loss: 0.9227 - val_acc: 0.7095
Epoch 93/100
8000/8000 [==============================] - 4s 465us/step - loss: 0.7576 - acc: 0.7820 - val_loss: 1.0209 - val_acc: 0.6740
Epoch 94/100
8000/8000 [==============================] - 4s 464us/step - loss: 0.7426 - acc: 0.7916 - val_loss: 0.9381 - val_acc: 0.6990
Epoch 95/100
8000/8000 [==============================] - 4s 461us/step - loss: 0.7657 - acc: 0.7803 - val_loss: 0.9433 - val_acc: 0.7040
Epoch 96/100
8000/8000 [==============================] - 4s 465us/step - loss: 0.7560 - acc: 0.7821 - val_loss: 0.9267 - val_acc: 0.7215
Epoch 97/100
8000/8000 [==============================] - 4s 463us/step - loss: 0.7044 - acc: 0.8065 - val_loss: 0.9255 - val_acc: 0.7095
Epoch 98/100
8000/8000 [==============================] - 4s 463us/step - loss: 0.7357 - acc: 0.7940 - val_loss: 0.9556 - val_acc: 0.7035
Epoch 99/100
8000/8000 [==============================] - 4s 459us/step - loss: 0.7017 - acc: 0.8055 - val_loss: 0.9035 - val_acc: 0.7330
Epoch 100/100
8000/8000 [==============================] - 4s 461us/step - loss: 0.7178 - acc: 0.8025 - val_loss: 0.8673 - val_acc: 0.7375
100/100 [==============================] - 1s 7ms/step
0.7999363780021668  0.79
0.79
0.9932187
0.7